# Imports 

In [1]:
# Descargar dataset desde Kaggle
import kagglehub
import numpy as np
import pandas as pd
import os
import shutil  
import warnings
# Importaciones para el manejo geoespacial
import geopandas as gpd
from shapely.geometry import Point
import subprocess 

# Desactivar advertencias de sjoin
warnings.filterwarnings('ignore', 'The CRS of the two GeoDataFrames are not the same. *')


e:\ANACONDA\envs\visualizacionAvanzada\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 🛠️ Utilidades


## Busqueda de distrito a partir de coordenadas

In [2]:
# --- PROCESAMIENTO GEOESPACIAL ---

def assign_borough_from_geometry(df: pd.DataFrame, lat_col: str, lon_col: str, output_col: str) -> pd.DataFrame:
    """
    Asigna el distrito (borough) a las coordenadas usando una unión espacial.
    
    Argumentos:
        df (pd.DataFrame): DataFrame de entrada.
        lat_col (str): Nombre de la columna de latitud.
        lon_col (str): Nombre de la columna de longitud.
        output_col (str): Nombre de la nueva columna de distrito a crear.
    """
    print(f"\n--- Iniciando asignación de distrito para {output_col} ---")
    
    # 1. Cargar límites de distritos de NYC (Boroughs)
    nyc_boundaries_url = "https://data.cityofnewyork.us/resource/gthc-hcne.geojson"
    
    try:
        boroughs_gdf = gpd.read_file(nyc_boundaries_url)
        
        boroughs_gdf = boroughs_gdf.set_geometry('geometry')

        # Renombrar la columna del distrito en el GeoDataFrame de límites
        boroughs_gdf.rename(columns={'boroname': 'borough_name_geo'}, inplace=True)
        
        if boroughs_gdf.crs is None:
             boroughs_gdf.set_crs(epsg=4326, inplace=True)
        else:
             boroughs_gdf = boroughs_gdf.to_crs(epsg=4326)

    except Exception as e:
        print(f"Error CRÍTICO al cargar o procesar los límites de los distritos: {e}")
        return df

    # 2. Convertir datos de Uber a GeoDataFrame
    df[lat_col] = pd.to_numeric(df[lat_col], errors='coerce')
    df[lon_col] = pd.to_numeric(df[lon_col], errors='coerce')
    
    df_clean = df.dropna(subset=[lat_col, lon_col]).copy()
    
    if df_clean.empty:
        print("No hay coordenadas válidas para procesar.")
        df[output_col] = 'Out of NYC/Unknown'
        return df

    geometry = [Point(xy) for xy in zip(df_clean[lon_col], df_clean[lat_col])]
    points_gdf = gpd.GeoDataFrame(df_clean, geometry=geometry, crs="EPSG:4326")
    
    # 3. Realizar Unión Espacial (Spatial Join)
    if points_gdf.crs != boroughs_gdf.crs:
        boroughs_gdf = boroughs_gdf.to_crs(points_gdf.crs)

    join_result = gpd.sjoin(points_gdf, boroughs_gdf[['borough_name_geo', 'geometry']], how="left", predicate="within")
    
    # 4. Mapear el resultado de nuevo al DataFrame original
    # Usar el nombre de columna dinámico (output_col)
    df[output_col] = join_result['borough_name_geo'].reindex(df.index)
    
    # 5. Rellenar valores faltantes (coordenadas fuera de NYC o inválidas)
    df[output_col] = df[output_col].fillna('Out of NYC/Unknown')
    
    print(f"--- Asignación de distrito ({output_col}) completada ---")
    print(f"Top 5 de distritos encontrados para {output_col}:")
    print(df[output_col].value_counts().head(5))
    
    return df


## Estimacion de la emision de CO2

In [3]:
# --- Supuestos ---
FUEL_DENSITY_KG_PER_L = 0.74      # masa por litro (gasolina), ~0.74 kg/L (ver fuente)
CO2_PER_LITRE_KG = 2.0844         # ejemplo (DEFRA condensed set): kg CO2 por litro de gasolina

# --- Funciones basadas en EMEP/COPERT (ejemplo para gasolina ligera) ---
def fuel_g_per_km_emep_petrol(V_kmh: float) -> float:
    """
    Fórmula de la guía EMEP/COPERT (ejemplo para coches <1.4 L en rango ~17.9-130 km/h).
    Devuelve consumo de combustible en gramos por km (g/km).
    Si la velocidad está fuera del rango razonable, aplicamos límites.
    Fuente: EMEP/EEA (COPERT) guidebook.
    """
    if np.isnan(V_kmh) or V_kmh <= 0:
        return np.nan
    # rango típico: si V < 10 o > 130 podemos saturar o usar una aproximación
    V = float(np.clip(V_kmh, 10.0, 130.0))
    # ecuación extraída (ver guía): consumo (g/km) = 81.1 - 1.014*V + 0.0068*V^2
    return 81.1 - 1.014 * V + 0.0068 * V**2

def co2_per_km_from_speed(V_kmh: float,
                          fuel_density_kg_per_l=FUEL_DENSITY_KG_PER_L,
                          co2_per_litre_kg=CO2_PER_LITRE_KG) -> float:
    """
    Calcula kg CO2 por km usando la ecuación de consumo + conversiones.
    """
    fuel_g_km = fuel_g_per_km_emep_petrol(V_kmh)   # g fuel / km (masa)
    if np.isnan(fuel_g_km):
        return np.nan
    fuel_kg_km = fuel_g_km / 1000.0               # kg fuel / km
    fuel_l_km = fuel_kg_km / fuel_density_kg_per_l  # L fuel / km
    co2_kg_km = fuel_l_km * co2_per_litre_kg
    return co2_kg_km

# --- Pipeline para un dataframe ---
def estimate_co2_dataframe(df: pd.DataFrame,
                           distance_col='trip_distance_km',
                           minutes_col='trip_minutes',
                           passengers_col='passenger_count'):
    df = df.copy()
    # velocidad media km/h
    df['avg_speed_kmh'] = df[distance_col] / (df[minutes_col] / 60.0)
    # consumo / emisiones por km
    df['fuel_g_per_km'] = df['avg_speed_kmh'].apply(fuel_g_per_km_emep_petrol)
    df['co2_kg_per_km'] = df['avg_speed_kmh'].apply(co2_per_km_from_speed)
    # total por viaje
    df['co2_kg_trip'] = df['co2_kg_per_km'] * df[distance_col]
    # por pasajero (si passenger_count==0 lo tratamos como 1)
    df['passenger_count_safe'] = df[passengers_col].replace({0:1}).fillna(1)
    df['co2_kg_per_passenger'] = df['co2_kg_trip'] / df['passenger_count_safe']
    return df

# 🔢 Proceso principal

In [4]:
# Descargar la versión más reciente
path = kagglehub.dataset_download("praveenluppunda/uber-dataset")
print("Path to dataset files:", path)

# Buscar archivo CSV en el directorio descargado
csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
if not csv_files:
    raise FileNotFoundError("No se encontró ningún archivo CSV en el dataset.")
    
file_path = os.path.join(path, csv_files[0])
print("Archivo CSV encontrado:", file_path)

# Cargar dataset
df = pd.read_csv(file_path)

100%|██████████| 3.90M/3.90M [00:00<00:00, 4.70MB/s]

Extracting files...


Path to dataset files: C:\Users\aesto\.cache\kagglehub\datasets\praveenluppunda\uber-dataset\versions\1
Archivo CSV encontrado: C:\Users\aesto\.cache\kagglehub\datasets\praveenluppunda\uber-dataset\versions\1\uber_data.csv


## Eliminar puntos con coordenadas (0,0)

In [5]:
# 0.1. FILTRO DE COORDENADAS (lat/lon ~ 0,0) 
# Primero, asegurar que las coordenadas son numéricas para el filtro
df['pickup_latitude'] = pd.to_numeric(df['pickup_latitude'], errors='coerce')
df['pickup_longitude'] = pd.to_numeric(df['pickup_longitude'], errors='coerce')
df['dropoff_latitude'] = pd.to_numeric(df['dropoff_latitude'], errors='coerce')
df['dropoff_longitude'] = pd.to_numeric(df['dropoff_longitude'], errors='coerce')

# Definir el umbral (e.g., +/- 0.1 grados de latitud y longitud)
ZERO_THRESHOLD = 0.1 

# Crear máscaras para identificar registros que tienen un pickup o dropoff cerca de (0,0)
is_pickup_zero = (abs(df['pickup_latitude']) < ZERO_THRESHOLD) & (abs(df['pickup_longitude']) < ZERO_THRESHOLD)
is_dropoff_zero = (abs(df['dropoff_latitude']) < ZERO_THRESHOLD) & (abs(df['dropoff_longitude']) < ZERO_THRESHOLD)

# Combinar las máscaras: filtrar si CUALQUIERA de las ubicaciones está cerca de (0,0)
df_clean = df[~(is_pickup_zero | is_dropoff_zero)].copy()

# Mostrar cuántos registros se eliminaron
rows_removed = len(df) - len(df_clean)
print(f"\n--- Limpieza de datos (Coordenadas ~0,0) ---")
print(f"Registros iniciales: {len(df)}")
print(f"Registros eliminados (cerca de 0,0): {rows_removed}")
print(f"Registros restantes: {len(df_clean)}")

df = df_clean # Quitamos los errores de coordenadas


--- Limpieza de datos (Coordenadas ~0,0) ---
Registros iniciales: 100000
Registros eliminados (cerca de 0,0): 1000
Registros restantes: 99000


## Obtener los distritos de salida y llegada

In [6]:

# 0.2. Asignar distrito (borough) a las coordenadas
# Asignar distrito de SALIDA
if 'pickup_latitude' in df.columns and 'pickup_longitude' in df.columns:
    df = assign_borough_from_geometry(df, 'pickup_latitude', 'pickup_longitude', 'pickup_borough')
    
# Asignar distrito de LLEGADA
if 'dropoff_latitude' in df.columns and 'dropoff_longitude' in df.columns:
    df = assign_borough_from_geometry(df, 'dropoff_latitude', 'dropoff_longitude', 'dropoff_borough')
    


--- Iniciando asignación de distrito para pickup_borough ---
--- Asignación de distrito (pickup_borough) completada ---
Top 5 de distritos encontrados para pickup_borough:
pickup_borough
Manhattan             90973
Queens                 5984
Brooklyn               1852
Bronx                   100
Out of NYC/Unknown       89
Name: count, dtype: int64

--- Iniciando asignación de distrito para dropoff_borough ---
--- Asignación de distrito (dropoff_borough) completada ---
Top 5 de distritos encontrados para dropoff_borough:
dropoff_borough
Manhattan             86560
Queens                 6774
Brooklyn               4429
Bronx                   711
Out of NYC/Unknown      505
Name: count, dtype: int64


## Otras transformaciones

Obtener el nombre de las características a partir de la tabla de información sobre los datos

In [7]:
# 1. Eliminar columna VendorID
if 'VendorID' in df.columns:
    df = df.drop(columns=['VendorID'])

# 2. Calcular minutos de viaje
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['trip_minutes'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# 3. Convertir distancia de millas a kilómetros
if 'trip_distance' in df.columns:
    df['trip_distance_km'] = df['trip_distance'] * 1.60934
    df = df.drop(columns=['trip_distance'])

# 4. Transformar RatecodeID a categorías
ratecode_map = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group ride",
    99: "Null/unknown"
}
if 'RatecodeID' in df.columns:
    df['RatecodeID'] = df['RatecodeID'].map(ratecode_map)

# 5. Eliminar columna store_and_fwd_flag
if 'store_and_fwd_flag' in df.columns:
    df = df.drop(columns=['store_and_fwd_flag'])

# 6. Transformar payment_type a categorías
payment_type_map = {
    0: "Flex Fare trip",
    1: "Credit card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip"
}
if 'payment_type' in df.columns:
    df['payment_type'] = df['payment_type'].map(payment_type_map)


## Calcular datos de emisiones de CO2
**Cómo se han calculado las emisiones de CO2:**


1. **Función de consumo de combustible dependiente de la velocidad**
    (Ejemplo basado en ecuación EMEP/COPERT para coches gasolina pequeños)

    $$
    \text{consumo\_combustible\_g\_por\_km} = 81.1 - 1.014 \cdot V + 0.0068 \cdot V^2
    $$

    Válida en el rango aproximado de **17.9–130 km/h** (ver guía EMEP/COPERT).  
    *Fuente: Agencia Europea del Medio Ambiente*

2. **Conversión de gramos de combustible a litros por kilómetro**

    $$
    \text{litros\_por\_km} = \frac{\text{consumo\_g\_por\_km} / 1000}{\rho_{\text{combustible}}}
    $$

    donde  
    $$
    \rho_{\text{gasolina}} \approx 0.74 \, \text{kg/L}
    $$

    *Fuente: toolkit.pops.int*

3. **Conversión de litros a kgCO₂ (según factor DEFRA)**

    $$
    \text{kgCO2\_por\_km} = \text{litros\_por\_km} \times \text{kgCO2\_por\_litro}
    $$

    Usar el valor exacto del **spreadsheet DEFRA** para el año y tipo de combustible correspondiente.  
    *Fuente: GOV.UK*

4. **Emisiones totales y por pasajero**

    $$
    \text{emisiones\_por\_viaje} = \text{kgCO2\_por\_km} \times \text{trip\_distance\_km}
    $$

    $$
    \text{emisiones\_por\_pasajero} = \frac{\text{emisiones\_por\_viaje}}{\max(1, \text{passenger\_count})}
    $$


In [8]:
df = estimate_co2_dataframe(df)

# 🗃️ Guardar resultados

In [9]:
# 7. Guardar el archivo final
output_file = "uber_dataset_con_distritos.csv"
df.to_csv(output_file, index=False)
print(f"\nArchivo procesado guardado como: {output_file}")

# 8. Eliminar el directorio temporal de descarga
shutil.rmtree(path)
print(f"Directorio temporal eliminado: {path}")


Archivo procesado guardado como: uber_dataset_con_distritos.csv
Directorio temporal eliminado: C:\Users\aesto\.cache\kagglehub\datasets\praveenluppunda\uber-dataset\versions\1


In [10]:
df['fecha'] = df['tpep_pickup_datetime'].dt.hour
df['fecha'].value_counts()

fecha
8     11606
9     10617
12    10421
13    10124
10     9538
11     9532
7      9480
0      6959
14     4825
1      4089
5      3639
2      2562
4      1888
6      1876
3      1844
Name: count, dtype: int64